# Q-Learning Experiments for Irrigation Scheduling

Notebook for running and experimenting with tabular Q-learning.

## Setup

In [ ]:
import numpy as np
from irrigation_env import IrrigationEnv
from irr_Qtable import train_q_learning, discretize_state, get_state_space_size, N_ACTIONS

In [ ]:
# Create environment instance
env = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 40.0),
    episode_length=90,
)

print(f"Environment created")
print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

## Training

In [ ]:
# Training parameters
n_episodes = 1000
alpha = 0.1
gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995
n_soil_bins = 12

print(f"Training Q-learning agent for {n_episodes} episodes...")
print(f"State space size: {get_state_space_size(n_soil_bins)}")
print(f"Action space size: {N_ACTIONS}")

In [ ]:
# Train Q-learning
Q = train_q_learning(
    env=env,
    n_episodes=n_episodes,
    alpha=alpha,
    gamma=gamma,
    epsilon_start=epsilon_start,
    epsilon_end=epsilon_end,
    epsilon_decay=epsilon_decay,
    n_soil_bins=n_soil_bins,
)

print("\nTraining complete!")
print(f"Q-table shape: {Q.shape}")
print(f"Non-zero entries: {np.count_nonzero(Q)}/{Q.size}")

## Inspection

In [ ]:
# Basic Q-table statistics
print("Q-table Statistics:")
print(f"Shape: {Q.shape}")
print(f"Min Q-value: {Q.min():.4f}")
print(f"Max Q-value: {Q.max():.4f}")
print(f"Mean Q-value: {Q.mean():.4f}")
print(f"Std Q-value: {Q.std():.4f}")
print(f"\nNon-zero entries: {np.count_nonzero(Q)}/{Q.size} ({100*np.count_nonzero(Q)/Q.size:.1f}%)")

In [ ]:
# Examine action preferences across all states
best_actions = np.argmax(Q, axis=1)
action_counts = np.bincount(best_actions, minlength=N_ACTIONS)

print("Action preferences (greedy policy):")
for action_idx, count in enumerate(action_counts):
    print(f"  Action {action_idx}: {count} states ({100*count/Q.shape[0]:.1f}%)")

In [ ]:
# Test learned policy on a single episode
obs, info = env.reset(seed=123)
total_reward = 0.0
done = False
step_count = 0

print("Testing learned policy:")
while not done and step_count < 10:
    state = discretize_state(obs, n_soil_bins)
    action = np.argmax(Q[state])
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    total_reward += reward
    
    if step_count < 5:
        print(f"  Step {step_count+1}: state={state}, action={action}, reward={reward:.3f}, SM={obs['soil_moisture'][0]:.3f}")
    
    step_count += 1

print(f"\nTotal reward (first {step_count} steps): {total_reward:.3f}")